In [22]:
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [23]:
# Data Source:https://www.kaggle.com/datasets/sarahquesnelle/canada-data
Temperature_DF = pd.read_csv("Resources/Canada_Temperature_Data.csv")

Temperature_DF2 = Temperature_DF.loc[(Temperature_DF["Year"] >= 1989) & (Temperature_DF["Year"] <= 2020), :]

Prov_DF = Temperature_DF2.loc[Temperature_DF2["Prov"].isin(["ON", "AB", "QC"]), :]

Prov_DF2 = Prov_DF.drop("Stn_Name", axis=1)

Prov_DF2


# Prov_DF2.to_csv("Canada_Temperature_Data2.csv", index=False)


,Year,Month,Prov,Tm,S,P
941030,1989,1,AB,-10.4,27.0,35.5
941031,1989,1,AB,-11.1,24.0,28.0
941032,1989,1,AB,-12.8,37.5,43.5
941033,1989,1,AB,-9.1,36.0,54.0
941034,1989,1,AB,-12.9,20.2,23.2
...,...,...,...,...,...,...
1357243,2017,9,QC,13.5,0.0,62.4
1357244,2017,9,QC,14.5,0.0,83.8
1357245,2017,9,QC,14.1,0.0,52.2
1357246,2017,9,QC,9.3,0.0,76.4


In [24]:
Canada_Temperature_DF = pd.read_csv("Resources/Canada_Temperature_Data2.csv")


Canada_Temperature_DF['Year'] = Canada_Temperature_DF['Year'].astype(str)


Canada_Temperature_DF.head(10)



,Year,Month,Seasons,Province,Average Temperature,Average Snowfall Recorded,Average Precipitation Recorded
0,1989,1,Winter,AB,-10.4,27.0,35.5
1,1989,1,Winter,AB,-11.1,24.0,28.0
2,1989,1,Winter,AB,-12.8,37.5,43.5
3,1989,1,Winter,AB,-9.1,36.0,54.0
4,1989,1,Winter,AB,-12.9,20.2,23.2
5,1989,1,Winter,AB,-9.4,29.8,45.6
6,1989,1,Winter,AB,-11.0,20.8,31.0
7,1989,1,Winter,AB,-9.5,47.9,62.7
8,1989,1,Winter,AB,-11.0,35.6,39.0
9,1989,1,Winter,AB,-11.9,29.4,44.4


In [29]:
# group by year, season, and province
grouped = Canada_Temperature_DF.groupby(['Year', 'Seasons', 'Province'])

# calculate summary statistics
summary_stats = grouped.agg({'Average Temperature': ['mean', 'median', 'std', 'var', 'sem'],
                             'Average Snowfall Recorded': ['mean', 'median', 'std', 'var', 'sem'],
                             'Average Precipitation Recorded': ['mean', 'median', 'std', 'var', 'sem']})

# flatten multi-level column names
summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]

# reset index
summary_stats = summary_stats.reset_index()


Prov_ON = summary_stats.loc[summary_stats["Province"].isin(["ON"]), :]
Prov_AB = summary_stats.loc[summary_stats["Province"].isin(["AB"]), :]
Prov_QC = summary_stats.loc[summary_stats["Province"].isin(["QC"]), :]

# print summary statistics by year, season, and province
print(Prov_AB)



     Year Seasons Province  Average Temperature_mean  \
0    1989  Autumn       AB                 -2.162500   
3    1989  Spring       AB                  9.460788   
6    1989  Summer       AB                 14.207393   
9    1989  Winter       AB                -11.405893   
12   1990  Autumn       AB                 -5.906480   
..    ...     ...      ...                       ...   
333  2016  Winter       AB                 -3.481915   
336  2017  Autumn       AB                 -3.464516   
339  2017  Spring       AB                 10.230769   
342  2017  Summer       AB                 15.566667   
345  2017  Winter       AB                 -7.524638   

     Average Temperature_median  Average Temperature_std  \
0                         -3.00                 5.608096   
3                          9.70                 4.161009   
6                         14.90                 3.297703   
9                        -11.30                 3.722552   
12                        -

In [50]:
# save summary_stats to Excel file
summary_stats.to_excel('summary_stats.xlsx', index=False)